In [58]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys
import os
import numpy as np
from tqdm import tqdm
import glob
import librosa
import pandas as pd

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from secs import SpeechObjectiveEvaluation

In [60]:
sr = 16000
models = ["resemblyzer", "wavlm", "wespeaker", "wespeaker_lm", "wespeaker_nolm"]
# models = ["resemblyzer"]
soe = SpeechObjectiveEvaluation(sr, models)

def cosine_similarity(e1, e2): # from wespeaker, delete the normalizing part
    e1 = torch.tensor(e1)
    e2 = torch.tensor(e2)
    cosine_score = torch.dot(e1, e2) / (torch.norm(e1) * torch.norm(e2))
    cosine_score = cosine_score.item()
    return cosine_score

Some weights of the model checkpoint at /mntcephfs/data/audiow/shoinoue/Model/hf_hub/wavlm/models--microsoft--wavlm-base-plus-sv/snapshots/feb593a6c23c1cc3d9510425c29b0a14d2b07b1e/ were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at /mntcephfs/data/audiow/shoinoue/Model/hf_hub/wavlm/models--microsoft--wavlm-base-plus-sv/snapshots/feb593a6c23c1cc3d9510425c29b0a1

Loaded the voice encoder model on cuda in 0.02 seconds.


In [61]:
base_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"
accent_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/wav/"
native_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/English/wav/"
fs = 16000

labels = {}
labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small"] = "only-gt_small"
labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating"] = "only-gt_large"
labels["VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating_small"] = "only-syn_small"
labels["VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating"] = "only-syn_large"
labels["VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating_small"] = "syn-gt_small"
labels["VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating"] = "syn-gt_large"

- Save

In [62]:
save = False
la = "Hindi"
tempfile = "temp.wav"


testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for mn in labels:
    print(mn)
    files = glob.glob(base_dir + mn + "/100000/wav/*.wav")
    files.sort()
    for path in tqdm(files):
        basename = os.path.basename(path)[:-4]
        accent = accent_dir + basename + ".wav"
        native = native_dir + basename + ".wav"
        save_path = "/".join(path.split("/")[:-2])+"/speaker_embeddings/" + basename + ".npy"
        if not(os.path.exists(save_path)) or save:
            data = soe.get_speaker_embedding(path)
            dirname = os.path.dirname(save_path)
            if not(os.path.exists(dirname)):
                os.makedirs(dirname, exist_ok=True)
            np.save(save_path, data)
            
        accent_path = f"./evaluation/speaker_embeddings/{la}/{basename}.npy"
        native_path = f"./evaluation/speaker_embeddings/English/{basename}.npy"
        paths = [accent_path, native_path]
        if not(os.path.exists(accent_path) and os.path.exists(native_path)) or save:
            for i, path in enumerate([accent, native]):
                save_path = paths[i]
                data = soe.get_speaker_embedding(path)
                dirname = os.path.dirname(save_path)
                if not(os.path.exists(dirname)):
                    os.makedirs(dirname, exist_ok=True)
                np.save(save_path, data)

VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small


100%|██████████| 105/105 [04:23<00:00,  2.51s/it]


VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating


100%|██████████| 105/105 [02:23<00:00,  1.37s/it]


VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating_small


...

100%|██████████| 105/105 [02:16<00:00,  1.30s/it]


VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating_small


100%|██████████| 100/100 [02:08<00:00,  1.29s/it]


VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating


100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


- Analysis

In [63]:
save = False
la = "Hindi"
scores = {}
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for mn in labels:
    scores[mn] = {}
    print(mn)
    files = glob.glob(base_dir + mn + "/100000/wav/*.wav")
    files.sort()
    initial = True
    for path in tqdm(files):
        basename = os.path.basename(path)[:-4]
        if not(basename in testsamples):
            continue
        pred_path = "/".join(path.split("/")[:-2])+"/speaker_embeddings/" + basename + ".npy"
        accent_path = f"./evaluation/speaker_embeddings/{la}/{basename}.npy"
        native_path = f"./evaluation/speaker_embeddings/English/{basename}.npy"
        pred = np.load(pred_path, allow_pickle=True).item()
        accent = np.load(accent_path, allow_pickle=True).item()
        native = np.load(native_path, allow_pickle=True).item()
        if initial:
            for key in pred:
                scores[mn][key] = []
            initial = False
        
        for key in pred:
            a = cosine_similarity(pred[key], accent[key])
            b = cosine_similarity(pred[key], native[key])
            diff = a-b
            scores[mn][key] += [diff]

VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small


100%|██████████| 105/105 [00:00<00:00, 452.85it/s]


VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating


100%|██████████| 105/105 [00:00<00:00, 504.99it/s]


VTN_fine-tuning_nocondition_syn2syn_melmel_hubert_norepeating_small


...

100%|██████████| 105/105 [00:00<00:00, 435.98it/s]


VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating_small


100%|██████████| 100/100 [00:00<00:00, 457.60it/s]


VTN_fine-tuning_nocondition_mix2syn_melmel_hubert_norepeating


100%|██████████| 100/100 [00:00<00:00, 564.66it/s]


In [73]:
for mn in list(labels.keys()):
    for key in scores[mn]:
        scores[mn][key] = np.mean(scores[mn][key])
df = pd.DataFrame(scores).T
df = np.round(df, 4)
df.index = list(labels.values())

In [74]:
df

,wavlm,wespeaker,wespeaker_lm,wespeaker_nolm,resemblyzer
only-gt_small,0.1347,0.1133,0.0794,0.0686,0.0328
only-gt_large,0.1331,0.0862,0.0638,0.0546,0.0393
only-syn_small,0.1380,0.0836,0.0617,0.0528,0.0302
only-syn_large,0.1310,0.0744,0.0645,0.0521,0.0378
syn-gt_small,0.1399,0.1499,0.0956,0.0847,0.0320
syn-gt_large,0.1381,0.1321,0.0831,0.0705,0.0427
